In [1]:
import numpy as np
import pandas as pd

In [25]:
data = pd.read_csv('AmesHousing.txt', delimiter='\t')
train = data[:1460]
test = data[1460:]


In [28]:
data_null_counts = train.isnull().sum()
data_null_counts_zero = data_null_sum[data_null_sum == 0]
df_no_mv = train[data_null_sum_zero.index]
df_no_mv.head()

Order        PID  MS SubClass MS Zoning  Lot Area Street Lot Shape  \
0      1  526301100           20        RL     31770   Pave       IR1   
1      2  526350040           20        RH     11622   Pave       Reg   
2      3  526351010           20        RL     14267   Pave       IR1   
3      4  526353030           20        RL     11160   Pave       Reg   
4      5  527105010           60        RL     13830   Pave       IR1   

  Land Contour Utilities Lot Config    ...     Enclosed Porch 3Ssn Porch  \
0          Lvl    AllPub     Corner    ...                  0          0   
1          Lvl    AllPub     Inside    ...                  0          0   
2          Lvl    AllPub     Corner    ...                  0          0   
3          Lvl    AllPub     Corner    ...                  0          0   
4          Lvl    AllPub     Inside    ...                  0          0   

  Screen Porch Pool Area Misc Val Mo Sold  Yr Sold  Sale Type  Sale Condition  \
0            0         0        0       5     2010        WD           Normal   
1          120         0        0       6     2010        WD           Normal   
2            0         0    12500       6     2010        WD           Normal   
3            0         0        0       4     2010        WD           Normal   
4            0         0        0       3     2010        WD           Normal   

   SalePrice  
0     215000  
1     105000  
2     172000  
3     244000  
4     189900  

[5 rows x 58 columns]

## pandas.Series.astype('category')
Some of the columns in the data frame df_no_mv contain string values. If these columns contain only a limited set of uniuqe values, they're known as categorical features.

We need to use the .cat accessor followed by the .codes property to actually access the underlying numerical representation of a column:

<b>train['Utilities'].cat.codes</b>

In [31]:
text_cols = df_no_mv.select_dtypes(['object']).columns

for col in text_cols:
    print(col+":", len(train[col].unique()))

MS Zoning: 6
Street: 2
Lot Shape: 4
Land Contour: 4
Utilities: 3
Lot Config: 5
Land Slope: 3
Neighborhood: 26
Condition 1: 9
Condition 2: 6
Bldg Type: 5
House Style: 8
Roof Style: 6
Roof Matl: 5
Exterior 1st: 14
Exterior 2nd: 16
Exter Qual: 4
Exter Cond: 5
Foundation: 6
Heating: 6
Heating QC: 4
Central Air: 2
Electrical: 4
Kitchen Qual: 5
Functional: 7
Paved Drive: 3
Sale Type: 9
Sale Condition: 5


In [32]:
for col in text_cols:
    train[col] = train[col].astype('category')

/Users/shiganyu/anaconda3/envs/dataquest/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [35]:
train['Utilities'].cat.codes.value_counts()

0    1457
2       2
1       1
dtype: int64

The drawback with this approach is that one of the assumptions of linear regression is violated here. Linear regression operates under the assumption that the features are linearly correlated with the target column. For a categorical feature, however, there's no actual numerical meaning to the categorical codes that pandas assigned for that colum. An increase in the Utilities column from 1 to 2 has no correlation value with the target column, and the categorical codes are instead used for uniqueness and exclusivity (the category associated with 0 is different than the one associated with 1).

The common solution is to use a technique called dummy coding. Instead of having a single column with n integer codes, we have n binary columns.

<b> pandas.get_dummies()</b>

In [37]:
dummy_cols = pd.DataFrame()
for col in text_cols:
    col_dummies = pd.get_dummies(train[col])
    np.concatenate([train, col_dummies], axis = 1)
    del train[col]

In [40]:
train.columns

Index(['Order', 'PID', 'MS SubClass', 'Lot Frontage', 'Lot Area', 'Alley',
       'Overall Qual', 'Overall Cond', 'Year Built', 'Year Remod/Add',
       'Mas Vnr Type', 'Mas Vnr Area', 'Bsmt Qual', 'Bsmt Cond',
       'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin SF 1', 'BsmtFin Type 2',
       'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF', '1st Flr SF',
       '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area', 'Bsmt Full Bath',
       'Bsmt Half Bath', 'Full Bath', 'Half Bath', 'Bedroom AbvGr',
       'Kitchen AbvGr', 'TotRms AbvGrd', 'Fireplaces', 'Fireplace Qu',
       'Garage Type', 'Garage Yr Blt', 'Garage Finish', 'Garage Cars',
       'Garage Area', 'Garage Qual', 'Garage Cond', 'Wood Deck SF',
       'Open Porch SF', 'Enclosed Porch', '3Ssn Porch', 'Screen Porch',
       'Pool Area', 'Pool QC', 'Fence', 'Misc Feature', 'Misc Val', 'Mo Sold',
       'Yr Sold', 'SalePrice'],
      dtype='object')

In [41]:
train['years_until_remod'] = train['Year Remod/Add'] - train['Year Built']

/Users/shiganyu/anaconda3/envs/dataquest/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.



## Missing Values
When values are missing in a column, there are two main approaches we can take:

<h3>Remove rows containing missing values for specific columns </h3>

<b>Pro:</b> Rows containing missing values are removed, leaving only clean data for modeling</br>

<b>Con:</b> Entire observations from the training set are removed, which can reduce overall prediction accuracy</br>

<h3>Impute (or replace) missing values using a descriptive statistic from the column</h3>

<b>Pro: </b>Missing values are replaced with potentially similar estimates, preserving the rest of the observation in the model.</br>

<b>Con:</b> Depending on the approach, we may be adding noisy data for the model to learn</br>


We'll focus on columns that contain at least 1 missing value but less than 365 missing values (or 25% of the number of rows in the training set). There's no strict threshold, and many people instead use a 50% cutoff (if half the values in a column are missing, it's automatically dropped). Having some domain knowledge can help with determining an acceptable cutoff value.

In [52]:
data_null_counts = train.isnull().sum()
data_null_select = data_null_counts[(data_null_counts > 0) & (data_null_counts < 584)]
df_missing_values = train[data_null_select.index]
df_missing_values.isnull().sum()
df_missing_values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 19 columns):
Lot Frontage      1211 non-null float64
Mas Vnr Type      1449 non-null object
Mas Vnr Area      1449 non-null float64
Bsmt Qual         1420 non-null object
Bsmt Cond         1420 non-null object
Bsmt Exposure     1419 non-null object
BsmtFin Type 1    1420 non-null object
BsmtFin SF 1      1459 non-null float64
BsmtFin Type 2    1419 non-null object
BsmtFin SF 2      1459 non-null float64
Bsmt Unf SF       1459 non-null float64
Total Bsmt SF     1459 non-null float64
Bsmt Full Bath    1459 non-null float64
Bsmt Half Bath    1459 non-null float64
Garage Type       1386 non-null object
Garage Yr Blt     1385 non-null float64
Garage Finish     1385 non-null object
Garage Qual       1385 non-null object
Garage Cond       1385 non-null object
dtypes: float64(9), object(10)
memory usage: 216.8+ KB


In [54]:
float_cols = df_missing_values.select_dtypes(['float64'])
float_cols.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 9 columns):
Lot Frontage      1211 non-null float64
Mas Vnr Area      1449 non-null float64
BsmtFin SF 1      1459 non-null float64
BsmtFin SF 2      1459 non-null float64
Bsmt Unf SF       1459 non-null float64
Total Bsmt SF     1459 non-null float64
Bsmt Full Bath    1459 non-null float64
Bsmt Half Bath    1459 non-null float64
Garage Yr Blt     1385 non-null float64
dtypes: float64(9)
memory usage: 102.7 KB


In [58]:
float_cols = float_cols.fillna(float_cols.mean())
float_cols.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 9 columns):
Lot Frontage      1460 non-null float64
Mas Vnr Area      1460 non-null float64
BsmtFin SF 1      1460 non-null float64
BsmtFin SF 2      1460 non-null float64
Bsmt Unf SF       1460 non-null float64
Total Bsmt SF     1460 non-null float64
Bsmt Full Bath    1460 non-null float64
Bsmt Half Bath    1460 non-null float64
Garage Yr Blt     1460 non-null float64
dtypes: float64(9)
memory usage: 102.7 KB
